# Geöffnete Skilifte absaugen



In [43]:
# Libraries laden
import requests
from bs4 import BeautifulSoup
import pandas as pd

## Teil 1: Liste mit allen Live-Skigebieten finden

In [44]:
startseite = requests.get("https://www.bergfex.ch/schweiz/").text
soup_startseite = BeautifulSoup(startseite, "html.parser")

Die Links heraussuchen

In [47]:
liveskigebiete = soup_startseite.find_all('li', {'style':'font-weight: bold;'})
linkliste = []

for gebiet in liveskigebiete:
    link = gebiet.find('a')['href']
    linkliste.append(link)

In [48]:
pd.DataFrame(linkliste).to_csv('skigebiete_linkliste.csv')

## Teil 2: Öffnungszeiten auslesen
Leere Liste starten

In [35]:
skigebiete = []

Seiten absaugen & souppieren

In [37]:
#website = requests.get("https://www.bergfex.ch/ybrig/").text
website = requests.get("https://www.bergfex.ch/engelberg/").text

In [39]:
soup = BeautifulSoup(website, "html.parser")

In [40]:
# Titel des Skigebiets
titel = soup.find('h1', {"class": "has-sup"})
titel = (titel.text.replace("Skigebiet ", ""))
# Offene Skilifte auslesen
for skigebiet in soup.find_all('table', class_="schneewerte"):
    offenelifte = (skigebiet.find_all('td')[3].text).split(" von ") # Auslesen im Format "12 von 24"
    total = offenelifte[1].strip()
    offen = offenelifte[0].strip()
    skigebiete.append({"titel": titel, "totallifte": total, "offen": offen}) # In die finale Liste laden
    
    
skigebiete

[{'titel': 'Hoch-Ybrig', 'totallifte': '12', 'offen': '1'},
 {'titel': 'Hoch-Ybrig', 'totallifte': '12', 'offen': '1'},
 {'titel': 'Engelberg Titlis', 'totallifte': '28', 'offen': '12'}]

## Ergänzen in einer Tabelle

## Alarmieren, wenn irgendetwas fehlschlägt

In [41]:
# Send E-Mail